## GEOG0027 (2020/1) Classification with Google Earth Engine (GEE)

This practical will use Google Earth Engine (GEE)'s python library [EE](https://developers.google.com/earth-engine) and [geemap library](https://geemap.org/) to automatically classify land covers and land uses in Shenzhen area. These two libraries are built to handle remote sensing (RS) data from the Cloud without physically downloading the data to our local computers, and they also allow easy python coding, where only small modifications are needed before handling large datasets. GEE hosts many popular RS datasets on the Cloud, and details of its data catalog can be found at: https://developers.google.com/earth-engine/datasets.

For the Shenzhen classification work, we start from displaying a basemap of the area.

In [1]:
import geemap, ee, os

Map = geemap.Map(center=[22.634, 114.19], zoom=9)
Map

Map(center=[22.634, 114.19], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(…

## Examining the time series
Let's define a rectangular region of interest, following [min lon, min lat, max mon, max lat] first, and display a short 'movie' (a .gif file in fact) of how this area has changed over the past decades.

In [2]:
shenzhen_rec = ee.Geometry.Rectangle([113.7659, 22.40, 114.6654, 22.8536]) 
Map.addLayer(shenzhen_rec, {}, 'AOI rec')
print(shenzhen_rec.getInfo())

{'type': 'Polygon', 'coordinates': [[[113.7659, 22.4], [114.6654, 22.4], [114.6654, 22.8536], [113.7659, 22.8536], [113.7659, 22.4]]]}


In [2]:
Map_gif = geemap.Map(center=[22.7511, 113.91], zoom=10)
Map_gif.add_landsat_ts_gif(roi=shenzhen_rec, start_year=1985, bands=['NIR', 'Red', 'Green'], frames_per_second=5)

{'type': 'Polygon', 'coordinates': [[[113.7659, 22.4], [114.6654, 22.4], [114.6654, 22.8536], [113.7659, 22.8536], [113.7659, 22.4]]]}
Generating URL...
Please wait ...
The GIF image has been saved to: /Users/qingling/Downloads/landsat_ts_mjf.gif
Adding animated text to GIF ...
Adding GIF to the map ...
The timelapse has been added to the map.


![shenzhengif](../../../../Downloads/landsat_ts_qfo.gif "shenzhen")

Next, we will compare such change by using a slider

In [3]:
landsat_ts = geemap.landsat_timeseries(roi=shenzhen_rec, start_year=1986, end_year=2020, \
                                       start_date='01-01', end_date='12-31')

layer_names = ['Landsat ' + str(year) for year in range(1986, 2021)]

geemap_landsat_vis = {
    'min': 0,
    'max': 3000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']} # You can change the vis bands here

Map2 = geemap.Map()
Map2.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, \
                 left_vis=geemap_landsat_vis, right_vis=geemap_landsat_vis)
Map2.centerObject(shenzhen_rec, zoom=10)
Map2

Map(center=[22.627302103068118, 114.2156500000001], controls=(WidgetControl(options=['position'], widget=Dropd…

We have manually defined a rectangle for Shenzhen area, but we can also use shape/vector files to select Areas of Interest. 

# Select 'Shenzhen' as the area of interest (AOI)
The vector border layer is imported from https://developers.google.com/earth-engine/datasets/tags/borders, which includes the [Global Administrative Unit Layers (GAUL) data](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level2) from 2015. You may notice that Shenzhen's boundary has expanded since (e.g. coastal landfill). We can manually draw some polygon and clip it to the GAUL border file, or, to make a simple example, we can add some 'buffer' (e.g. 3000 meters) to the GAUL boundary data. This inevitably will introduce some areas outside the border of Shenzhen, e.g. part of Hong Kong, so you can work out some more elegant way to combining/clipping multiple mask layers if time allows. Justify you choice and summarize how you made it in the report. In the following example, I will use the `shenzhen_buffer` as my AOI.

In [3]:
cities = ee.FeatureCollection("FAO/GAUL/2015/level2")
#Map.addLayer(cities, {}, 'Cities', False)

shenzhen = cities.filter(ee.Filter.eq('ADM2_NAME', 'Shenzhen'))
outline = ee.Image().byte().paint(**{
  'featureCollection': shenzhen,
  'color': 1,
  'width': 3
})
Map.addLayer(outline, {}, 'Shenzhen')

# Next, add some buffer to include the coastal expansion areas
shenzhen_buffer = ee.Geometry(shenzhen.geometry()).buffer(3000)
Map.addLayer(shenzhen_buffer, {}, 'Buffer around Shenzhen')
#Map.addLayer(rec, "Original rec bounds")
Map

Map(bottom=57372.0, center=[22.634, 114.19], controls=(WidgetControl(options=['position'], widget=HBox(childre…

Next, let's load some Landsat images for the Shenzhen area. I've defined here a python function called `display_landsat_collection` to do so. It automatically loads both the [surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR) and [annual NDVI](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_ANNUAL_NDVI) image collections from GEE's data catalog and also calculates the annual means for each band. 

In order to run such function, you will need to supply a year (any year since 1984) and an AOI. In the following example, I chose year 2019 and the same Shenzhen buffer to demonstrate.

In [5]:
# to read in the Landsat data collection(s)

def load_landsat_collection(year, aoi, cloud_tolerance = 3.0, 
                            DISPLAY_ON_MAP = False, MEDIAN_ONLY=False):
    '''This function allows GEE to display an image collection 
    that fall within the cloud tolerance, e.g. 3.0%.
    There are two optional flag:
    When DISPLAY_ON_MAP is TRUE, display this layer onto Map;
    When MEDIAN_ONLY is True, only median SR is loaded into landsat_ts. 
    MEDIAN_ONLY is faster than loading all collections. 
    '''
    if year >= 2013:
        layer_name = 'LC08' # LS8: 2013-now        
        fcc_bands = ['B5', 'B4', 'B3']
    elif year == 2012: # # LS7: 1999- , however SLT error >= 1999:
        layer_name = 'LE07' 
        fcc_bands = ['B4', 'B3', 'B2']
    elif year >=1984:
        layer_name = 'LT05' # LS5: 1984-2012
        fcc_bands = ['B4', 'B3', 'B2']
    else:
        print('Please name a year after 1984')
        
    collection_name_sr = f"LANDSAT/{layer_name}/C01/T1_SR" 
    # You can also use the following line, if interested in incorperating ndvi
    collection_name_ndvi = f"LANDSAT/{layer_name}/C01/T1_ANNUAL_NDVI" 

    if not(MEDIAN_ONLY):
        all_sr_image = ee.ImageCollection(collection_name_sr) \
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31') \
            .filter(ee.Filter.lt('CLOUD_COVER', cloud_tolerance))\
            .sort('system:time_start') \
            .select('B[1-7]') \
            .sort('CLOUD_COVER')

        # reduce all_sr_image to annual average per pixel
        mean_image = all_sr_image.mean()
        mean_image = mean_image.clip(aoi).unmask()

        ndvi_image = ee.ImageCollection(collection_name_ndvi)\
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31')\
            .select('NDVI')\
            .first()
        ndvi_image = ndvi_image.clip(aoi).unmask()

        #mean_image.addBands(ndvi_image, 'NDVI')

    median_image = landsat_ts.filterDate(f'{year}-01-01', f'{year}-12-31').first()
    median_image = median_image.clip(aoi).unmask()
    
    if DISPLAY_ON_MAP == True:
        
        landsat_vis_param = {
            'min': 0,
            'max': 3000,
            'bands': fcc_bands 
        }
        geemap_landsat_vis = {
            'min': 0,
            'max': 3000,
            'bands': ['NIR', 'Red', 'Green']
        }
        ndvi_colorized_vis = {
            'min': 0.0,
            'max': 1.0,
            'palette': [
            'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
            '012E01', '011D01', '011301']
        }
        
        if not(MEDIAN_ONLY):
            Map.addLayer(ndvi_image, ndvi_colorized_vis, 'NDVI '+str(year),  opacity=0.9)
            Map.addLayer(mean_image, landsat_vis_param, "Mean Ref "+str(year))
        Map.addLayer(median_image, geemap_landsat_vis, "Median Ref "+str(year))

    if MEDIAN_ONLY:
        return median_image
    else:
        return all_sr_image, mean_image, median_image, ndvi_image 

landsat_ts = geemap.landsat_timeseries(roi=shenzhen_buffer, start_year=1984, end_year=2020, \
                                       start_date='01-01', end_date='12-31')


Map = geemap.Map(center=[22.634, 114.19], zoom=10)
# All you need to modify is the YEAR below:
all_image_2019, mean_2019, median_2019, ndvi_2019 = load_landsat_collection(2019,\
                                        shenzhen_buffer, cloud_tolerance = 3,\
                                        DISPLAY_ON_MAP = True)
Map

Map(center=[22.634, 114.19], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(…

We can also check the metadata from the Landsat image collection we just loaded from the Cloud. Have a look of the output. Any useful information?

In [11]:
first_image = all_image_2019.first() 

props = geemap.image_props(first_image)
print( props.getInfo())
#print(first_image.get('IMAGE_DATE').getInfo())
#print(first_image.get('CLOUD_COVER').getInfo(), '%')

{'CLOUD_COVER': 0.21, 'CLOUD_COVER_LAND': 0.23, 'EARTH_SUN_DISTANCE': 0.989463, 'ESPA_VERSION': '2_23_0_1b', 'GEOMETRIC_RMSE_MODEL': 7.889, 'GEOMETRIC_RMSE_MODEL_X': 5.66, 'GEOMETRIC_RMSE_MODEL_Y': 5.495, 'IMAGE_DATE': '2019-11-14', 'IMAGE_QUALITY_OLI': 9, 'IMAGE_QUALITY_TIRS': 9, 'LANDSAT_ID': 'LC08_L1TP_122044_20191114_20191202_01_T1', 'LEVEL1_PRODUCTION_DATE': 1575301122000, 'NOMINAL_SCALE': 30, 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0', 'SATELLITE': 'LANDSAT_8', 'SENSING_TIME': '2019-11-14T02:52:28.2228010Z', 'SOLAR_AZIMUTH_ANGLE': 153.700989, 'SOLAR_ZENITH_ANGLE': 45.366386, 'SR_APP_VERSION': 'LaSRC_1.3.0', 'WRS_PATH': 122, 'WRS_ROW': 44, 'system:asset_size': '627.230926 MB', 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'], 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_122044_20191114', 'system:index': 'LC08_122044_20191114', 'system:time_end': '2019-11-14 02:52:28', 'system:time_start': '2019-11-14 02:52:28', 'system:version': 1576236040308713}


Next, examine the mean, median surface reflectance and/or NDVI layers we've visualized (switching the layers on and off, calculating the differences, etc.) . Which one is better? Which band should we include into the classification?

In [7]:
NIR = mean_2019.select('B5')
Red = mean_2019.select('B4')
mean_ndvi = NIR.subtract(Red).divide(NIR.add(Red))

NIR = median_2019.select('NIR')
Red = median_2019.select('Red')
median_ndvi = NIR.subtract(Red).divide(NIR.add(Red))

Map.addLayer(mean_ndvi.subtract(median_ndvi), {'min': 0.0,'max': 0.2}, 'Diff in NDVI')
#Map

# Nextly, let's run some unsupervised classification (e.g. K-means) with geemap



## Unsupervised classification
Let's start with the 2019 Median image as an example. in the following code cell, a function called `unsupervised_classifier` has been defined to classify an image by take several input parameters (or 'arguments'). You can don't have to worry about the detail of the code except the very first line, where information about the 'arguments' can be found.

In [17]:
def unsupervised_classifier(image, aoi, n_clusters=5, output_filename='', DISPLAY_ON_MAP = False):
    '''This function provides a simple K-means classifier,
    with a default no. of cluster of 5. User will need to specify 
    an AOI and an image to be classified.
    Optional arguments:
    n_clusters defines the number of clusters in the K-means classifier;
    output_filename should be a quoted string, e.g. 'Shenzhen_Landsat_Kmeans_2019.tif';
    DISPLAY_ON_MAP can be switched on, so the cluster map will be added to Map.
    '''
    
    # Make the training dataset:
    training_points = image.sample(**{
        'region': aoi,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })

    #Map.addLayer(training_points, {}, 'training points', False) # No need to visualise this layer

    # Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training_points)

    # Cluster the input using the trained clusterer.
    class_result = image.cluster(clusterer)
    
    # Reclassify the map to avoid zero, in case of masking. E.g. from [0, 1, 2, 3, 4] to [1, 2, 3, 4, 5]
    #class_result = class_result.remap(list(range(0, n_clusters)), list(range(1, n_clusters+1)))

    if DISPLAY_ON_MAP:
        # Display the clusters with random colors.
        Map.addLayer(class_result.randomVisualizer(), {}, 'Clusters '+output_filename[-8:-4], opacity=1.0)

    if output_filename == '':
        print('Classification finished. No output exported.')
    else:
        #Export the result directly to your computer/Hub:
        geemap.ee_export_image(class_result, filename=output_filename, \
                            scale=900, region=aoi, file_per_band=False) # When scale is small, GEE won't allow download due to size

    return class_result

The above cell only 'defines' a function but does NOT execute it, so we now need to actually run it by calling the function name with appropriate arguments:

In [25]:
class_result = unsupervised_classifier(mean_2019, shenzhen_buffer, n_clusters=5, DISPLAY_ON_MAP = True)
Map

Classification finished. No output exported.


Map(bottom=114444.0, center=[22.634, 114.19], controls=(WidgetControl(options=['position'], widget=HBox(childr…

## Next, we will need to define or name these unsupervised classes
You need to carefully compare the classified results with the original images to decide which cluster belongs to what class. Then, you can name the classes and assign appropriate RGB colours accordingly. However, the classified clusters may differ between different years, so you may need to change the legend for some of the years. Also, bare in mind, there might be mis-classified pixels. How can you improve the results?

In [26]:
legend_keys = ['Urban', 'Vegetation', 'Bright Urban',  'Water', 'Shaded Vegetation']
legend_colors = ['#FFFFB3', '#8DD3C7', '#FFFFB0','#80B1D3', '#8DD3C0']

Map.addLayer(class_result, {'min': 0, 'max': 4, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

These are the numbers needed for later modelling work. We will also show you how to export these values automatically later. Now, let's have a close examination of the classification results. 

For the year 2019 with 5 clusters, the results seem to include two 'Urban' and two 'Vegetation' clusters. In cases similar this, we may consider grouping multiple clusters together, by re-mapping the cluster numbers:

In [27]:
# Reclassify the map. You may want to avoid zero, in case of masking.
remapped_class_result = class_result.remap([0, 1, 2, 3, 4], [1, 2, 1, 3, 2])

legend_keys = ['Urban', 'Vegetation', 'Water']
legend_colors = ['#FFFFB3', '#8DD3C7','#80B1D3']

Map.addLayer(remapped_class_result, {'min': 1, 'max': 3, 'palette': legend_colors}, 'Remapped clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

However, sometimes you may find the reclustered results less satisfactory, so consider carefully before executing the remaping code.

After remapping the clusters, we have just three final clusters now.  Would this be the same result if we simply run the `unsupervised_classifier` function with `n_clusters = 3`? Have a try. The number of clusters to use depends on the image, and it may different between different years.

## Cluster areas
Next, we need to find out the size of each cluster, i.e. how many pixels belong to each class, and the total size of each class (hint: how big is each pixel?)

In [28]:
import numpy

def calculate_class_size(class_result, year, legend_keys, PRINT_STATS=False):
    n_clusters = len(legend_keys)
    
    stats = {'Year': year}
    for i in range(n_clusters):
        remap = numpy.zeros(n_clusters)
        remap[i] = 1
        class_0 = class_result.remap(list(range(0, n_clusters)), list(remap))
        class_stats0 = geemap.image_stats(class_0, scale=30)
        #print(class_stats0)
        sum_class0 = class_stats0.getInfo()['sum']
        sum_clean = int(sum_class0['remapped'])
        if PRINT_STATS:
            print(legend_keys[i], 'has', sum_clean, 'pixles.')
        stats[legend_keys[i]] = sum_clean
        
    return stats

legend_keys = ['Urban', 'Vegetation', 'Bright Urban',  'Water', 'Shaded Vegetation']        
stats_2019 = calculate_class_size(class_result, 2019, legend_keys, PRINT_STATS=True)

Urban has 1024540 pixles.
Vegetation has 813730 pixles.
Bright Urban has 367348 pixles.
Water has 899544 pixles.
Shaded Vegetation has 653759 pixles.


# Repeat for multiple years
By now, you should be able to understand that the `load_landsat_collection` function allows us to just vary the 'year' variable (and leave AOI and Cloud Tolerance constant) to loop through a time series of data. Similarly, for the `unsupervised_classifier` function, we can vary the input 'image' variable (an output from the `load_landsat_collection` function) in order classify multiple images. So, below is a loop built for running K-means classification for a time series of Landsat for the Shenzhen area. Years between 1987 to 2016 are selected to match our socio-economic data. When this cell is running (you can spot a star key on the LHS of the cell), you can track the progress by looking at the output messages. Once the cell finished running, an integer number will be shown outside of the LHS of the cell, and you can examine the layers in the Map (if the DISPLAY flag was set to True)

In [30]:
cluster_pixels = []

for year in range(1987, 2017, 10): # Only to demonstrate every 10 years, but you should run every year 
    
    median_image = load_landsat_collection(year, shenzhen_buffer, cloud_tolerance=3,\
                                          MEDIAN_ONLY = True)
    class_result = unsupervised_classifier(median_image, shenzhen_buffer, \
                    n_clusters=5, output_filename=f'Shenzhen_Landsat_Kmeans_{year}.tif',\
                    DISPLAY_ON_MAP = True)
    
    legend_keys = ['Urban', 'Class 2', 'Class 3',  'Class 4', 'Class 5'] #change the class names
    stats = calculate_class_size(class_result, year, legend_keys)
    print(stats)
    
    #if 'Urban' in stats: print(f'There are {int(stats["Urban"])} urban pixles in {str(year)}.')
    
    cluster_pixels.append(numpy.fromiter(stats.values(), dtype=int))

#Map

Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/UCL/Teaching-2020/GEE/Shenzhen_Landsat_Kmeans_1987.tif
{'Year': 1987, 'Urban': 1232, 'Class 2': 1072284, 'Class 3': 423519, 'Class 4': 669464, 'Class 5': 1592380}
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/UCL/Teaching-2020/GEE/Shenzhen_Landsat_Kmeans_1997.tif
{'Year': 1997, 'Urban': 1138527, 'Class 2': 1050767, 'Class 3': 409195, 'Class 4': 606934, 'Class 5': 553497}
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/UCL/Teaching-2020/GEE/Shenzhen_Landsat_Kmeans_2007.tif
{'Year': 2007, 'Urban': 991944, 'Class 2': 815590, 'Class 3': 350030, 'Class 4': 848970, 'Class 5': 752385}


After running the above loop of code to classify a time series of Landsat images, all cluster areas (i.e. pixel counts) have been recorded into the `cluster_pixels` variable, and cluster layers are mapped into the Map.

## Export results to excel (.csv) files
Lastly, we will use the following code to export the sizes of each class/cluster into a csv file, and you will need this file for modeling in the next part of the coursework.

In [31]:
header = 'Year, ' + ', '.join( legend_keys )
numpy.savetxt("Shenzhen_pixel_stats.csv", cluster_pixels, delimiter=",", header=header)       

[array([   1987,    1232, 1072284,  423519,  669464, 1592380]), array([   1997, 1138527, 1050767,  409195,  606934,  553497]), array([  2007, 991944, 815590, 350030, 848970, 752385])]


# Congratulations (on finishing the classification part)
Now you should be able to open the CSV file in excel or R to continue with the [modeling part](https://github.com/qwu-570/GEOG0027_Coursework/blob/2020-2021/docs/6_UrbanModel.ipynb) of the coursework.